In [ ]:
import os
import torch

import numpy as np
import dgl.function as fn
import torch.nn as nn
import torch.nn.functional as F
import scanpy as sc
import networkx as nx
import matplotlib.pyplot as plt

import graph
import model
import visualization

rng = np.random.default_rng()

In [ ]:
ANNDATA_DIR = 'annData'
DLPFC_ANNDATA_DIR = os.path.join(ANNDATA_DIR, '1.DLPFC')

In [ ]:
test_sample_id = '151507'
sample = sc.read_h5ad(os.path.join(DLPFC_ANNDATA_DIR, "{}.h5ad".format(test_sample_id)))
sample

In [ ]:
# test = torch.Tensor([[1,2,3,4,5],[6,7,8,9,10]])
# tn = F.normalize(test)
# print(torch.sum(tn**2, dim=1))


In [ ]:
# Create a DGL graph object
g = graph.create_dgl_graph(sample)

In [ ]:
# sample the graph
g = graph.create_random_dgl_subgraph(g, 20)

In [ ]:
g.nodes()

In [ ]:
g.edges()

In [ ]:
g.edges(form='all')

In [ ]:
g.ndata['x']

In [ ]:
f_norm = F.normalize(g.ndata['x'], p=2, dim=1)
cos_sim = torch.mm(f_norm, f_norm.t())
# 1 if the vectors are similar, 0 if not
cos_sim

In [ ]:
# 0 if the vectors are similar, 1 if not
cos_dist = 1 - cos_sim
cos_dist

In [ ]:
torch.arange(g.num_nodes())

In [ ]:
weight_updates = cos_dist[g.edges()]
weight_updates

In [ ]:
cos_dist

In [ ]:
g.edges()

In [ ]:
# similarity = ot.dist(g.ndata['x'], metric='cosine')
# mins, maxs = torch.min(similarity), torch.max(similarity)
# ranges = maxs - mins
# similarity = (similarity - mins)/ranges
# similarity = 1 - similarity - torch.eye(g.num_nodes())
# similarity


In [ ]:
g.apply_edges(model.update_edge_weights)

In [ ]:
def train_vis(g, model, sg_nids, pos):
    features = {
        'x': g.ndata['x'],
        'w': g.edata['w']
    }
    g.ndata['embedding'] = g.ndata['x']

    # initialize graph for visualization

    for e in range(20000):
        features = model(g, features)
        g.ndata['x'] = features['x']
        g.edata['w'] = features['w']
        
        # todo update network attributes.

            # nx.draw_networkx_labels(network, pos=subgraph.ndata['coords'].numpy())
        if e % 1000 == 0:
            sg = graph.create_random_dgl_subgraph(g, nids=sg_nids)
            g_vis = visualization.sample_networkx_from_dgl(sg)
            network_weights, minw, maxw, rangew = visualization.get_network_weights(g_vis)
            pos = nx.spring_layout(g_vis, weight='w', threshold=1e-7)
            nx.draw(g_vis, pos=pos, node_size=10, width=0.3, node_color='red', edge_color=network_weights, edge_vmin=0, edge_vmax=1,  edge_cmap=plt.cm.gist_yarg)
            plt.show()

In [ ]:
sg_nids = rng.choice(g.num_nodes(), 20, replace=False)
sg = graph.create_random_dgl_subgraph(g, nids=sg_nids)
pos = visualization.get_initial_dgl_graph_positions(sg)
g_vis = visualization.sample_networkx_from_dgl(sg)

In [ ]:
network_weights, minw, maxw, rangew = visualization.get_network_weights(g_vis)
nx.draw(g_vis, pos=pos, node_size=10, width=0.3, node_color='red', edge_color=network_weights, edge_vmin=minw, edge_vmax=maxw,  edge_cmap=plt.cm.gist_yarg)

In [ ]:
m = model.Model()
train_vis(g, m, sg_nids, pos)

In [ ]:
g.update_all(message_func=model.send_neighborhood_data, reduce_func=model.reduce_neighborhood_score)

In [ ]:
g.apply_edges(model.update_edge_weights)